# Google Trends API for Python
This Google notebook Colab (Collaboratory ) is part of the repository that delivers the supplementary material for the paper: **Disruptive Solutions on Requirement Engineering for Agile Software Development: A tertiary study**. The entire repository is available at https://doi.org/10.5281/zenodo.6864813

In [ ]:
#Mount path for original data and for results
from google.colab import drive
import os

drive.mount('/content/drive')

#Path to the original data. The data used in this study are available in Zenodo's repository.
data_path =os.path.join("/content/drive/MyDrive/Colab Notebooks", "DisruptiveSolutionsData", "zenodo", "solutions-only-for-search-google-trends.csv")

#Path to the search results data on Google Trends.
trends_path = os.path.join("/content/drive/MyDrive/Colab Notebooks", "DisruptiveSolutionsData/csv", "trends.csv")


#This notebook is prepared to search on Google Trends (search_trends = True) or use previously searched results (search_trends = False). 
search_trends = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import the original data to this notebook.
import random
from collections import OrderedDict
import pandas as pd

# Load the dataset into a pandas dataframe.
df_sheet_family, df_hyper, df_tmp = None, None, None

#solution-family (name of the solution family). key (terms to be searched on Google Trends). solution (name of the solutions)
column_names = ["solution-family", "key", "solution"]
df_families = pd.DataFrame(columns = column_names)
df_families.reset_index

df_sheet_family = pd.read_csv(data_path, sep=";") 

#The original data present four columns with keys (Key_01, Key_02, Key_03, Key_04), although this routine is prepered to work with n keys. Let us put this four columns in only one in a new Dataframe. 
key_col = range(1, 4)

for n in key_col:
  if 1 <= n <= 9:
    key_tmp = "%s%s" % ("Key_0", n)
  else:
    key_tmp = "%s%s" % ("Key_", n)

  df_tmp =  df_sheet_family[['solution-family', key_tmp, 'Solution-Cluster']] 
  df_tmp.rename(columns = {key_tmp:'key', 'Solution-Cluster' : 'solution'}, inplace = True)

  df_families = df_families.append(df_tmp.dropna(), ignore_index=True)

#Delete any row if the new key column is empty (NaN). Sort by the solution-family column.
df_hyper = df_families.dropna()
df_hyper = df_hyper.sort_values('solution-family', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
df_hyper.reset_index

#Delete Dataframe that will no longer be used.
del df_sheet_family



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#If you set the variable search_trends to True, this cell will install the pytrends API.

if search_trends:
  !pip install pytrends

In [ ]:
#If you set the variable search_trends to True, this cell will handle the search in google trends and create the CSV file in trends_path
if search_trends:
  from pandas._libs.lib import is_string_array
  import time
  import math
  from pytrends.request import TrendReq

  pytrend = TrendReq()

  keys = []
  #Extract key to a list with no duplications
  keys= list(OrderedDict.fromkeys(df_hyper['key']))

  df_trends = None
  ikey = 0
  
  key_start = 0
  key_end = key_start + 0
  key_step = "%s_%s" % (key_start, key_end)
  
  for key in keys:
    if key != math.nan:
      #Uncoment the code below for comparative search with the term 'Requirements engineering'
      #kw_list=[key, 'Requirements engineering']
      kw_list=[key]
      #Set the search period. Although the period of this study is from 2015 to 2021, we prefer a more extensive search for testing. 
      #cat=1227 means the Computer Science category for searching
      pytrend.build_payload(kw_list, cat=1227, timeframe='2000-01-01 2022-12-31', geo='', gprop='')
      df = pytrend.interest_over_time()
      if df.empty:
        #Creates a list of lists with default values to indicate that the key was not found.
        data = [['1900-01-01', 0]]
        # Create the pandas DataFrame
        df = pd.DataFrame(data, columns=['date', key])

      if ikey == 0:
        df_trends = df
      else:
        df_trends[key] = df[key]
      
      ikey = ikey + 1

      if ikey % 10 == 0:
        time.sleep(60) # The routine will sleep for 60 seconds each 10 iterations. To avoid Google Trends 403 or 409 errors (the account has exceeded the quota).
      else:
        time.sleep(3) # The routine will sleep for 3 seconds each iteration. To avoid Google Trends 403 or 409 errors.

  #Write the results to a CSV and delete the Dataframe. It will open again.
  df_trends.to_csv(trends_path)
  del df_trends


In [ ]:
#This cell consolidates google trend results from months to years, and across families.
from pandas.core.frame import DataFrame
from pandas._libs.lib import is_string_array
import time
import math

#The following code reads the Google Trends results and reopens the df_trends Dataframe.
df_trends = pd.read_csv(trends_path)

#The following code creates the new Dataframe df_trends_family_year [YEAR, Family] 
df_year, df_trends_year, df_trends_family_year = None, None, None
tmp_data = []

#Agregate trends by year
df_trends['date'] = pd.to_datetime(df_trends['date'])
df_trends['YEAR'] = df_trends['date'].dt.year
df_trends_year=df_trends.groupby('YEAR').agg(['sum']).reset_index()
df_trends_year.reset_index
years = list(df_trends_year['YEAR'])

head_trends = list(df_trends.columns)

for family_idx, family_row in df_families.iterrows(): #iterate over families
  for count, year in enumerate(years): #iterate over years
    #Search the value of key/year in df_trends_year
    key_value= family_row['key']
    key_column= family_row['key']
    df = df_trends_year.loc[df_trends_year['YEAR'] == year]
    count_value = df[key_value].iloc[0]['sum']
    family_value = family_row['solution-family']

    dict2 = {'YEAR': year, 'Family': family_value, 'Count': count_value}
    tmp_data.append(dict2)

df_trends_family_year = pd.DataFrame(tmp_data)
df_trends_family_year_agregated=df_trends_family_year.groupby(['YEAR', 'Family']).agg(['sum']).reset_index()
df_trends_family_year_agregated.reset_index
df_trends_family_year_pivot = df_trends_family_year_agregated.pivot(index='YEAR', columns='Family')['Count']

#After aggregation, the name of columns became a tuple. 
#The following code extracts the real name of the column
df_trends_family_year_pivot.columns = ['{}'.format(x[1]) for x in df_trends_family_year_pivot.columns]
heads = list(df_trends_family_year_pivot.columns)
df_trends_family_year_pivot = df_trends_family_year_pivot.rename_axis('YEAR').reset_index()


In [ ]:
#Google Trends does not work with absolute values. It works with results from zero to 100.  
#Zero represents the period with less popularity, and 100 represents the period with more popularity of some terms. 
#But, in the aggregation routine above, we lost this pattern. This procedure will normalize the aggregated results between zero and 100.
def normalize_0_100(dfNNorm):
    result = dfNNorm.copy()
    for feature_name in dfNNorm.columns:
        if feature_name != 'YEAR':
          max_value = dfNNorm[feature_name].max()
          min_value = dfNNorm[feature_name].min()
          if (max_value - min_value) != 0:
            result[feature_name] = ((dfNNorm[feature_name] - min_value) / (max_value - min_value)) * 100
    return result

In [ ]:
#This routine removes outliers that can get in the way of the analysis. Values above three standard deviations will be excluded.
from scipy import stats
def remove_outlier(df):
    result = df.copy()
    result = result[(np.abs(stats.zscore(result)) < 3).all(axis=1)]
    return result

In [ ]:
#The model to classify the solution as disruptive works with the concepts of slope and intercept of a regression curve. 
#This procedure calculates the slope and intercepts for every family or solution.
def calc_slope_and_intercept(dfSlope, xLabel, yLabel, initialyear, finalYear):
    dfSlope = dfSlope.query("YEAR >= @initialyear and YEAR <= @finalYear")
    x = dfSlope[xLabel]
    x.reset_index
    x -= initialyear

    y = dfSlope[yLabel]

    n = np.size(x)
  
    x_mean = np.mean(x)
    y_mean = np.mean(y)
    x_mean,y_mean
      
    Sxy = np.sum(x*y)- n*x_mean*y_mean
    Sxx = np.sum(x*x)-n*x_mean*x_mean
      
    slope = Sxy/Sxx
    intercept = y_mean-slope*x_mean

    return [slope, intercept, yLabel]

In [ ]:
#Plot the graphs for solution families
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

initialYear = 2011
finalYear = 2021

column_names = ["slope", "interception", "solution-family"]
df_slope_intercept = pd.DataFrame(columns = column_names)
df_slope_intercept.reset_index   

heads = list(df_trends_family_year_pivot.columns)
heads.append("YEAR")
#df_trends_family_year_pivot = df_trends_family_year_pivot.reindex(columns=heads)

sns.set_theme(color_codes=True)

#df_trends_family_year_pivot_tmp = df_trends_family_year_pivot.query("YEAR >= @initialYear and YEAR <= @finalYear")
df_normalized = normalize_0_100(df_trends_family_year_pivot.query("YEAR >= @initialYear and YEAR <= @finalYear"))
df_normalized.reset_index

for head in heads:
  sns.color_palette("husl", 8)
  fig = plt.figure()
  if head != 'YEAR':
    leg = list([head])

    df_normalized_with_out_outliner = remove_outlier(df_normalized[['YEAR', head]])

    slope_and_intercept = calc_slope_and_intercept(df_normalized_with_out_outliner.copy(), 'YEAR', head, initialYear, finalYear)
    tx = "%s%s%s%s" % ("Slope: ", round(slope_and_intercept[0], 4), " / Intercept: ", round(slope_and_intercept[1], 4))

    # Acumulate slope and intercept in a df
    df_slope_intercept.loc[len(df_slope_intercept)] = slope_and_intercept

    intercept_treshold = 17.990916

    #Uncomment the "if" below according to interest
#    if slope_and_intercept[0] > 0 and slope_and_intercept[1] < intercept_treshold: #PLOT DISRUPTIVE FAMILIES
#    if slope_and_intercept[0] > 0 and slope_and_intercept[1] >= intercept_treshold: # PLOT NOT DISRUPTIVE SOLUTIONS - POSITIVE SLOPE, BUT INTERCEPTION GRATER THAN OR IGUAL THE intercept_treshold
    if slope_and_intercept[0] <= 0: # PLOT NO DISRUPTIVE FAMILIES - NEGATIVE OR 0 SLOPE
#    if 1 == 1: # PLOT ALL FAMILIES
      fig.text(.2, .88, tx)
      g= sns.regplot(x='YEAR', y= head , data=df_normalized_with_out_outliner)
      fig.legend(labels=leg)
      g.set(xlabel='Year', ylabel='Trends (Hype)')    
      g.set(ylim=(-20, 105), xlim=(initialYear, finalYear + 0.5))

#We first execute this cell and identify the interception mean. In our study it was 17.990916
#df_positive_slope=df_slope_intercept.query("slope > 0")
#df_positive_slope.reset_index
#df_positive_slope.describe()


,slope,interception
count,12.000000,12.000000
mean,5.781875,18.023918
std,3.270365,20.387139
min,1.241062,-18.181818
25%,2.868667,3.210227
50%,5.799626,18.453011
75%,8.118615,31.307296
max,10.739394,48.605689


In [ ]:

#Information about the itercepts
df2=df_slope_intercept.query("slope > 0")
df2.reset_index
df2['interception'].describe()
#df2
#df_slope_intercept['interception'].describe()
#df_slope_intercept['interception'].quantile(q=0.50)

count    12.000000
mean     18.023918
std      20.387139
min     -18.181818
25%       3.210227
50%      18.453011
75%      31.307296
max      48.605689
Name: interception, dtype: float64

In [ ]:
#This cell consolidates the results of google trends from months to years and in SOLUTIONS
from pandas.core.frame import DataFrame
#Create new dataframe df_trends_family_year [YEAR, Family] 
from pandas._libs.lib import is_string_array
import time
import math
df_trends_solution_year = None
tmp_data = []


#The agregate trends by year is already done

years = list(df_trends_year['YEAR'])

head_trends = list(df_trends.columns)

for solution_idx, solution_row in df_families.iterrows(): # iterate over families
  for count, year in enumerate(years): #iterate of years
    #Search the value of key/year in df_trends_year
    key_value= solution_row['key']
    key_column= solution_row['key']
    df = df_trends_year.loc[df_trends_year['YEAR'] == year]
    count_value = df[key_value].iloc[0]['sum']
    solution_value = solution_row['solution']

    df2 = {'YEAR': year, 'solution': solution_value, 'Count': count_value}
    tmp_data.append(df2)

df_trends_solution_year = pd.DataFrame(tmp_data)
df_trends_solution_year=df_trends_solution_year.groupby(['YEAR', 'solution']).agg(['sum']).reset_index()
df_trends_solution_year.reset_index
df_trends_solution_year_pivot = df_trends_solution_year.pivot(index='YEAR', columns= 'solution')['Count']

#After agregation, the name of columns became a tuple. 
#Extract the real name of the column
df_trends_solution_year_pivot.columns = ['{}'.format(x[1]) for x in df_trends_solution_year_pivot.columns]

df_trends_solution_year_pivot = df_trends_solution_year_pivot.rename_axis('YEAR').reset_index()


In [ ]:
#Plot solutions for demand in only one graph
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


df_trends_year_new = df_trends_year.copy()
df_trends_year_new.columns = ['{}'.format(x[0]) for x in df_trends_year_new.columns]
headsTmp = (["Machine Learning", "Design Thinking", "Canvas"])
#headsTmp = (["Abuser story", "Adaptative Random Testing", "Analytic Hierarchy"])
#headsTmp = (["Automatic Runtime Reappraisal of Weights", "Automatically validated against the storytests", "Brainstorming"])
#headsTmp = (["Cumulative Voting", "Delivery stories", "Ethnography", "Expert judgment"])
#headsTmp = (["Focal group", "Framework of prioritization", "Frequent prioritization", "Goal Question Metric"])
#headsTmp = (["Index cards", "Interview", "Joint Application Development", "Meeting"])
#headsTmp = (["Numerical Assignment", "Prioritization", "Prioritization of Stories", "Product grooming"])
#headsTmp = (["Prototyping", "Questionnaire", "Retrospectives", "Story boarding"])
#headsTmp = (["Story Card", "Story points", "Test case prioritization", "Testing before coding"])
#headsTmp = (["Use case", "User stories", "Continuous Testing Frame", "WORKSHOP"])
#headsTmp = (['Delivery stories', 'Prioritization of Stories', 'Product grooming', 'Story boarding', 'Story Card', 'Story points', 'User stories'])

heads = headsTmp.copy()
heads.append("YEAR")

sns.set_theme(color_codes=True)

fig = plt.figure()

for head in heads:
  sns.color_palette("husl", 8)
  if head != 'YEAR':
    df_normalized = df_trends_year_new[['YEAR', head]]
    df_normalized = normalize_0_100(df_trends_year_new)
    df_normalized = df_normalized.query("YEAR >= @initialYear")
    slope_and_intercept = calc_slope_and_intercept(df_normalized.copy(), 'YEAR', head, initialYear, finalYear)
    tx = "%s%s%s%s" % ("Slope: ", round(slope_and_intercept[0], 4), " / Intercept: ", round(slope_and_intercept[1], 4))

    g = sns.regplot(x='YEAR', y= head , data=df_normalized)

fig.legend(labels=headsTmp)
fig.text(.2, .88, tx)

g.set(xlabel='Year', ylabel='Trends (Hype)')
plt.show()




---

